# **0-  Load Liberary**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install librosa
!pip install resampy

In [ ]:
import os
import pathlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import IPython.display as ipd
import librosa
import librosa.display
from IPython import display
from IPython.display import Audio

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# **1 - Load Data**


In [ ]:
Path_to_file='/content/drive/MyDrive/Colab Notebooks/TestComandRecognitio/dataset_wMFCC_13_Subband_10_augmentation.csv'
Commands = pd.read_csv(Path_to_file)
Commands.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,Commands
0,-432.497345,108.525978,-46.043198,49.915398,18.001318,-15.540771,-15.885378,-19.559963,-5.000407,-11.663811,...,6400.230661,4542.971538,4706.768984,4354.761470,5013.935653,5250.210596,577.547157,5811.918673,4745.430481,comm_1
1,-386.088623,87.918175,5.496625,35.695213,3.503410,7.033689,-9.529474,-5.634079,-1.617541,2.154280,...,5566.044938,5092.406127,5358.857797,5003.461747,5700.656899,4421.390943,550.173450,5861.900427,4598.814817,comm_1
2,-368.412317,76.847614,7.507425,32.031477,3.698797,5.856561,-7.837803,-4.993077,-1.869171,1.921389,...,5552.031207,5175.249686,5387.091683,5140.601056,5623.690722,4952.317604,5448.877063,5542.568738,5475.302725,comm_1
3,-408.819916,87.582993,4.758339,38.538712,2.175874,5.975663,-8.784441,-6.605148,-1.656542,2.222985,...,5532.837592,5082.927138,5324.122851,4870.374491,5797.126987,4365.193602,660.419463,5647.066258,4743.809009,comm_1
4,-409.638916,81.466248,7.327656,33.718288,3.090757,3.360864,-7.855571,-7.146142,-1.728623,2.185934,...,5554.922223,5178.251574,5379.345030,4920.129293,5757.099664,4390.073950,629.854808,5647.039233,4739.627126,comm_1


#**2 - test Train Split**


In [ ]:
X = Commands.iloc[: ,:-1].values
Y = Commands['Commands'].values

In [ ]:
# As this is a multiclass classification problem onehotencoding our Y
from sklearn.preprocessing import StandardScaler, OneHotEncoder
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [ ]:
print(Y.shape)
X.shape

(11217, 20)


(11217, 23)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=42,test_size=0.2, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((8973, 23), (8973, 20), (2244, 23), (2244, 20))

In [ ]:
# scaling our data with sklearn's Standard scaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((8973, 23), (8973, 20), (2244, 23), (2244, 20))

In [ ]:
#Reshape for CNN MODEL
x_traincnn =np.expand_dims(x_train, axis=2)
x_testcnn= np.expand_dims(x_test, axis=2)
x_traincnn.shape, y_train.shape, x_testcnn.shape, y_test.shape

((8973, 23, 1), (8973, 20), (2244, 23, 1), (2244, 20))

# **1 - CNN Model**


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
model_checkpoint = ModelCheckpoint('Model_CNN.h5', monitor='val_accuracy', save_best_only=True)
early_stop=EarlyStopping(monitor='val_accuracy',mode='auto',patience=5,restore_best_weights=True)
lr_reduction=ReduceLROnPlateau(monitor='val_accuracy',patience=5,verbose=1,factor=0.2,min_lr=0.00001)

In [ ]:
import tensorflow.keras.layers as L

model = tf.keras.Sequential([
    L.Conv1D(512,kernel_size=5, strides=1,padding='same', activation='relu',input_shape=(x_traincnn.shape[1],1)),
    L.BatchNormalization(),
    L.MaxPool1D(pool_size=5,strides=2,padding='same'),

    L.Conv1D(256,kernel_size=5,strides=1,padding='same',activation='relu'),
    L.BatchNormalization(),
    L.MaxPool1D(pool_size=5,strides=2,padding='same'),

    L.Conv1D(128,kernel_size=3,strides=1,padding='same',activation='relu'),
    L.BatchNormalization(),
    L.MaxPool1D(pool_size=3,strides=2,padding='same'),

    L.Flatten(),
    L.Dense(20,activation='softmax')
])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 13, 512)           3072      
                                                                 
 batch_normalization (Batch  (None, 13, 512)           2048      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 7, 512)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 7, 256)            655616    
                                                                 
 batch_normalization_1 (Bat  (None, 7, 256)            1024      
 chNormalization)                                                
                                                        

In [ ]:
history=model.fit(x_traincnn, y_train, epochs=40, validation_data=(x_testcnn, y_test),
                  batch_size=32,callbacks=[early_stop,lr_reduction,model_checkpoint])

# **2 - LSTM Model**

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=42,test_size=0.2, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((2244, 13), (2244, 20), (561, 13), (561, 20))

In [ ]:
# scaling our data with sklearn's Standard scaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((2244, 13), (2244, 20), (561, 13), (561, 20))

In [ ]:
#reshape for lstm
X_train_reshaped = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
X_test_reshaped = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
X_train_reshaped.shape, X_test_reshaped.shape, x_test.shape, y_test.shape

((2244, 13, 1), (561, 13, 1), (561, 13), (561, 20))

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
model_checkpoint = ModelCheckpoint('Model_LSTM.h5', monitor='val_accuracy', save_best_only=True)
early_stop=EarlyStopping(monitor='val_accuracy',mode='auto',patience=5,restore_best_weights=True)
lr_reduction=ReduceLROnPlateau(monitor='val_accuracy',patience=5,verbose=1,factor=0.2,min_lr=0.00001)

from keras.layers import LSTM,BatchNormalization
model01=Sequential()
model01.add(LSTM(256,return_sequences=True,input_shape=(X_train_reshaped.shape[1], 1)))
model01.add(Dropout(0.2))
model01.add(LSTM(128,return_sequences=True))
model01.add(Dropout(0.2))
model01.add(LSTM(64))
model01.add(Dropout(0.2))
model01.add(Flatten())
model01.add(Dense(20,activation = 'softmax'))
model01.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model01.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 13, 256)           264192    
                                                                 
 dropout (Dropout)           (None, 13, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 13, 128)           197120    
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 flatten_1 (Flatten)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 20)                1300      
                                                                 
Total params: 512020 (1.95 MB)
Trainable params: 51202

In [ ]:
hist=model01.fit(X_train_reshaped, y_train, epochs=40, validation_data=(X_test_reshaped, y_test),batch_size=32, verbose=1,
               callbacks=[early_stop,lr_reduction,model_checkpoint])

# **3 - BI-LSTM Model**


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
model_checkpoint = ModelCheckpoint('Model_BiLSTM.h5', monitor='val_accuracy', save_best_only=True)
early_stop=EarlyStopping(monitor='val_accuracy',mode='auto',patience=5,restore_best_weights=True)
lr_reduction=ReduceLROnPlateau(monitor='val_accuracy',patience=5,verbose=1,factor=0.2,min_lr=0.00001)
import tensorflow.keras.layers as L
model = tf.keras.Sequential([

    L.Bidirectional(L.LSTM(256,return_sequences=True),input_shape=(X_test_reshaped.shape[1], 1)),
    L.BatchNormalization(),

    L.Bidirectional(L.LSTM(128,return_sequences=True)),
    L.BatchNormalization(),

    L.Bidirectional(L.LSTM(64,return_sequences=True)),
    L.BatchNormalization(),

    L.Flatten(),
    L.Dense(20,activation='softmax')
])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_3 (Bidirecti  (None, 13, 512)           528384    
 onal)                                                           
                                                                 
 batch_normalization_6 (Bat  (None, 13, 512)           2048      
 chNormalization)                                                
                                                                 
 bidirectional_4 (Bidirecti  (None, 13, 256)           656384    
 onal)                                                           
                                                                 
 batch_normalization_7 (Bat  (None, 13, 256)           1024      
 chNormalization)                                                
                                                                 
 bidirectional_5 (Bidirecti  (None, 13, 128)          

In [ ]:
hist=model01.fit(X_train_reshaped, y_train, epochs=40, validation_data=(X_test_reshaped, y_test),batch_size=32, verbose=1,
               callbacks=[early_stop,lr_reduction,model_checkpoint])

# **4 - CNN-Bi-LSTM-DNN Model**


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
model_checkpoint = ModelCheckpoint('Model_CNN_Bi_LSTM_DNN.h5', monitor='val_accuracy', save_best_only=True)
early_stop=EarlyStopping(monitor='val_accuracy',mode='auto',patience=5,restore_best_weights=True)
lr_reduction=ReduceLROnPlateau(monitor='val_accuracy',patience=5,verbose=1,factor=0.2,min_lr=0.00001)
import tensorflow.keras.layers as L

model = tf.keras.Sequential([
    L.Conv1D(256,kernel_size=5, strides=1,padding='same', activation='relu',input_shape=(x_traincnn.shape[1],1)),
    L.BatchNormalization(),
    L.MaxPool1D(pool_size=5,strides=2,padding='same'),
    L.Dropout(0.2),

    L.Conv1D(128,kernel_size=5,strides=1,padding='same',activation='relu'),
    L.BatchNormalization(),
    L.MaxPool1D(pool_size=5,strides=2,padding='same'),
    L.Dropout(0.2),

    L.Conv1D(128,kernel_size=5,strides=1,padding='same',activation='relu'),
    L.BatchNormalization(),
    L.MaxPool1D(pool_size=5,strides=2,padding='same'),

    L.Bidirectional(L.LSTM(64,return_sequences=True)),
    L.BatchNormalization(),
    L.Dropout(0.2),

    L.Bidirectional(L.LSTM(64,return_sequences=True)),
    L.BatchNormalization(),
    L.Dropout(0.2),

    L.Flatten(),
    L.Dense(128,activation='relu'),
    L.Dense(20,activation='softmax')
])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 23, 256)           1536      
                                                                 
 batch_normalization_24 (Ba  (None, 23, 256)           1024      
 tchNormalization)                                               
                                                                 
 max_pooling1d_12 (MaxPooli  (None, 12, 256)           0         
 ng1D)                                                           
                                                                 
 dropout_13 (Dropout)        (None, 12, 256)           0         
                                                                 
 conv1d_13 (Conv1D)          (None, 12, 128)           163968    
                                                                 
 batch_normalization_25 (Ba  (None, 12, 128)          

In [ ]:
history=model.fit(x_traincnn, y_train, epochs=40, validation_data=(x_testcnn, y_test),
                  batch_size=32,callbacks=[lr_reduction,model_checkpoint])

**Command Classification Rates for the CNN-BiLSTM-DNN Model with 13 WMFCC and 10 Subband Spectral Centroid  Features on the MACD Dataset, with Data augmentations FOR 40 epochs**

In [ ]:
epochs = [i for i in range(40)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
test_acc = history.history['val_accuracy']
test_loss = history.history['val_loss']

fig.set_size_inches(20,6)
ax[0].plot(epochs , train_loss , label = 'Training Loss')
ax[0].plot(epochs , test_loss , label = 'Validation Loss')
ax[0].set_title('Training & Validation Loss')
ax[0].legend()
ax[0].set_xlabel("Epochs")

ax[1].plot(epochs , train_acc , label = 'Training Accuracy')
ax[1].plot(epochs , test_acc , label = 'Validation Accuracy')
ax[1].set_title('Training & Validation Accuracy')
ax[1].legend()
ax[1].set_xlabel("Epochs")
plt.show()